# Aqua 0.7 Operator Refactor
_5-Dec-19, donny@, ..._

## Purpose
To improve the transparency and ease of understanding of Qiskit’s operator logic and algorithmic usage. Specifically, to reconcile with the Terra operator hierarchy and make the ExpectationValue and Evolution algorithms more visible, explicit, and extensible.

## Present State (Aqua 6.1) and Opportunities
The representation of exponential matrices sparsely as linear combinations of Pauli operators is critical in many quantum algorithms. As such, the Operator classes are the workhorses of Aqua, containing both the expectation value and evolution logic used by most of the algorithms in Aqua.

However, there are several opportunities for improvement:
* **Operator Algorithms**
    * **Ease of understanding** - The "Operator algorithm" logic - the ExpectationValue, Evolution, grouping, and symmetry analysis - is spread across the 3000-line operator hierarchy, and is very branchy for different modes of execution
    * **Ease of extension** - Modification to the expectation value, evolution, grouping, and symmetry logic is a core use case (e.g. the [CVaR expectation](https://arxiv.org/abs/1907.04769), [linar combination evolution](https://arxiv.org/abs/1202.5822), or the many recent papers on [pauli grouping](https://www.nature.com/articles/nature23879)).
* **WeightedPauli General (Non-Aqua) Usage**
    * The WeightedPauli operator logic is broadly useful, and can be moved into Terra for usage beyond Aqua.
    * WeightedPauli's ExpectationValue may be better placed in Ignis.
* **Remote Operator Algorithms** - Aer's fast ExpectationValue is not transparently or cleanly interchangeable with Aqua's local ExpectationValue today. The concept of an Algorithm not provided by Aqua is not yet defined to support this type of interchangeability cleanly.

### Operator Algorithms Ease of Use

#### Vanilla Expectation Example

Aqua's ExpectationValue is not contained within a single function or module, but rather split into several functions without a clear interface or flow for user usage. This is due to structural constraints in Aqua which are no longer present, where the algorithm requiring the expectation value held the backend object and could run circuits, but the operator could not. Presently, the code to take an expectation value in VQE is too long to paste here, but we encourage the reader to scan lines [361-395 of Aqua 6.1 VQE’s](https://github.com/Qiskit/qiskit-aqua/blob/stable/qiskit/aqua/algorithms/adaptive/vqe/vqe.py#L361) ExpectationValue calculation to try to understand where and how the expectation is computed. We’ve been asked by numerous Aqua users to explain how this code works, and most do not attempt to use it on their own.

The following is the shortest possible way to write an expectation value in Aqua. Note that it fundamentally requires the user to understand a certain execution flow, the correct functions to use to do this, and how those functions work with their execute mode. This takes a few hours to understand at least, often days. Further, there are no hints that tomography is being performed here, or matrix multiplication if the system chooses to do that instead.

In [1]:
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.components.variational_forms import RY
from qiskit.quantum_info import Pauli
from qiskit import BasicAer, execute, QuantumCircuit
from qiskit.circuit import Parameter
qasm_sim = BasicAer.get_backend('qasm_simulator')

In [2]:
op = WeightedPauliOperator([
    [.5, Pauli.from_label('IX')],
    [.2j, Pauli.from_label('ZY')],
])
circuit = QuantumCircuit(2)
circuit.h([0,1])

evaluation_circuits = op.construct_evaluation_circuit(wave_function=circuit, statevector_mode=False)
result = execute(evaluation_circuits, qasm_sim).result()
expect, std = op.evaluate_with_result(result=result, statevector_mode=False)
expect

(0.5+0.005078125j)

#### Vanilla Evolution Example

Evolution is somewhat more succinct, but 

In [4]:
from qiskit.circuit import Parameter

In [5]:
op = WeightedPauliOperator([
    [.5, Pauli.from_label('IX')],
    [.2, Pauli.from_label('ZY')],
])
circuit = QuantumCircuit(2)

θ = Parameter('θ')
instr = op.evolve_instruction(evo_time=θ)
circuit.append(instr, [0,1])
print(circuit.draw(fold=4000))
print('Decomposed:')
circuit.decompose().draw(fold=4000)

        ┌─────────────────┐
q_0: |0>┤0                ├
        │  Evolution^1(θ) │
q_1: |0>┤1                ├
        └─────────────────┘
Decomposed:


#### Tasks Not Possible Today

* Explicitly using Aer's ExpectationValue, or explicitly disabling it
* Modifying the ExpectationValue, Evolution, grouping, or symmetry code without copying a significant amount of code

### WeightedPauliOperator Not Available in Terra

Terra contains operator utilities in the `qiskit.quantum_info.operators` directory, focused primarily on the mathematical operations between matrix-represented operators of various types (e.g. Choi, Pauli, etc.). Terra's operators do not contain much of the Pauli-basis logic in Aqua today, and are not interoptable with Aqua's operator algorithms. As such, these utilities are only accessible to Aqua users.

### Ignis and Aer Logic Living in Aqua

* Aqua's ExpectationValue relies on a form of tomography, which in principle should live within Ignis's tomography utilities.
* Aer's ExpectationValue is a super-superuser feature today
    * The interface (and existence) of the Aer-provided algorithm is not made obvious because there is no way to specify a non-Aqua-provided algorithm in Qiskit
    * In Aqua, there is no simple way to specify which ExpectationValue algorithm the user wants. 
    * Aer's ExpectationValue is woven throughout the core operator code in a way that is branchy, inexorable, and difficult for users to understand and control.

## Core Requirements / Interface

1. Broadly Available (Qiskit-wide) WeightedPauli Logic
    1. Basic Construction and Usage
    1. Interface, Variants, Organization
        1. Option A: WeightedPauli class in Terra
        1. Option B: WeightedUnitary class in Terra
    1. Integration Testing
1. ExpectationValue and Evolution algorithm hierarchies
    1. Interface, Variants, Organization
        1. Usage of Operator Abstractions within Aqua Algorithms
        1. Usage of Operator Abstractions without Aqua Algorithms
    1. ExpectationValue Reconciliation with Ignis tomography
        1. Option A: Leave ExpectationValue in Aqua
        1. Option B: Keep ExpectationValue algo in Aqua, move logic into Ignis
        1. Option C: Move ExpectationValue algo into Ignis
1. Aer ExpectationValue and Evolution
    1. Option I: Leave AerExpectation in Aqua
    1. Option II: Central Qiskit Algorithms base classes
    1. Option III: Merge Aqua and Terra
1. Smooth Borders with Applications Primitives - e.g. FermionicOperator
    1. Move Operator Translation Into Application Stacks, Consistently

### Broadly Available (Qiskit-wide) WeightedPauli Logic

A **WeightedPauliOperator** is an n-qubit operator represented sparesely as a complex combination of n-qubit Pauli operators (tensor products of n single-qubit Paulis). All hermitian operators can be represented in this basis.

Many quantum algorithms depend on this sparse representation of operators in the Pauli basis. Terra contains a suite of Matrix-represented operators within the quantum_info module, including rich logic for combining and manipulating various operator types, but does not include a WeightedPauli operator. 
* The Aqua operator does not need to replicate Terra's logic within MatrixOperator, and should instead rely on Terra for Matrix-based operator logic. 
* Aqua is the not the only place in Qiskit which requires WeightedPauli Operators. These are used across quantum information, and should be interoptable with the other existing Terra operators.

**Requirement:** The mathematical (non-algorithmic) Operator logic in Aqua should be moved into Terra, and Aqua should use these as its core operator objects.

In [ ]:
from qiskit.quantum_info.operators import WeightedPauliOperator

#### Basic Construction and Usage

Today, Aqua supports two modes for construction of a WeightedPauli operator:

In [6]:
op = WeightedPauliOperator([
    [.5, Pauli.from_label('IX')],
    [.2, Pauli.from_label('ZY')],
])

In [7]:
op = WeightedPauliOperator.from_list(
    paulis=[Pauli.from_label('IX'),
            Pauli.from_label('ZY')],
    weights=[.5, .2])

The above code lacks the mathematical structure or power of typical operator expressions. As such, Qiskit should support the following:
##### Mathematical Construction

In [2]:
from qiskit.quantum_info.operators.pauli import X, Y, Z, I
from qiskit.quantum_info.operators.projectors import zero, one, plus, minus

##### From Circuit Operator Construction

In [ ]:
circ

#### Grouping

#### Integration Testing

### Breaking out the ExpectationValue and Evolution to be explicit modules which can be interchanged and extended.

Users want to easily find and understand ExpectationValue and Evolution logic, and understand the algorithmic variations available to them.

...

In [ ]:
from qiskit.aqua.algorithms import ExpectationValue, Evolution

avg = ExpectationValue(operator, backend).run(circuit)
instr = Evolution(operator, backend).instruction(param)

### Terse and transparent Operator usage within the Aqua algorithms, characteristic of end-user usage of the same algorithms.

...

In [ ]:
class VQE(QuantumAlgorithm):
    def __init__(self, operator, var_form, optimizer,
                 initial_point=None, backend=backend, callback=None, ...):
        ...
        self._expectation_value = ExpectationValue(self._operator, self._backend)
    
    def _energy_evaluation(self, params):
        circuits = self._var_form.construct_circuit(params)
        energy, stdev = self._expectation_value.run(circuits)
        return energy

### Explicit modules for Aer's ExpectationValue and Evolution which can be swapped for Aqua's, and a framework for remote backends to provide RemoteAlgorithm modules without depending on Aqua. 

...

In [ ]:
from qiskit.providers.aer import AerExpectationValue

my_vqe = VQE()

### Reconciling the implicit tomography in Aqua's expectation value with Ignis's tomography functionality to avoid code duplication and allow generic access to the functionality.

...

### Clear and smooth borders with the Operator-related objects in Aqua, such as the FermionicOperator and Hamiltonian in Chemistry, and the proposed LadderOperator and OptimizationProblem.

...

## Technical Design

* Changes to Terra Operator Hierarchy
* Aqua Algorithm Hierarchy
    * Groups and Symmetries Options
* Changes to Ignis
* Qiskit Remote Algorithms
* Aer ExpectationValue and Evolution
* IBMQ or other backend RemoteAlgorithms

### Changes to Terra Operator Hierarchy

There are two options for the migration of the Operator mathematical logic into Terra's quantum_info:

#### Option A: Move WeightedPauliOperator into quantum_info and refactor

...

#### Option B: Add Linear Combination functionality to Operator to allow for WeightedPauli, WeightedClifford, and Combinations

* e.g. Pauli a + Pauli b = LinComb(a, b)
* Circuit Operator?

...

...

## Timeline and Gameplan

...

## Future Extension Opportunities